# FLAN T 5 IPO Training

Minimal implementation of Implicit Preference Optimization (IPO) training for FLAN T 5 model.

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
import optax
import json
from typing import Dict, List, Tuple
from transformers import AutoTokenizer
from flax.training import train_state
import wandb
import os
from tqdm import tqdm
import numpy as np

# Initialize TPU
try:
    import tensorflow as tf
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU initialized successfully")
except:
    print("TPU initialization failed, falling back to CPU/GPU")

class PreferenceDataset:
    def __init__(self, json_path):
        with open(json_path, 'r') as f:
            raw_data = json.load(f)

        self.data = []
        for question, entry in raw_data.items():
            for hop, hop_data in entry["hops"].items():
                queries = hop_data["queries"]
                preferences = hop_data["preference_pairs"]
                for i, j in preferences:
                    self.data.append({
                        "question": question,
                        "preferred": queries[i],
                        "dispreferred": queries[j]
                    })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
    def get_batch(self, batch_size, rng_key):
        """Get a random batch for JAX training"""
        indices = random.choice(rng_key, len(self.data), (batch_size,), replace=False)
        batch = [self.data[int(idx)] for idx in indices]
        return batch

In [ ]:
# Install required packages for TPU
!pip install -q flax transformers[flax] optax

from transformers import FlaxT5ForConditionalGeneration, T5Config
from flax import linen as nn
import jax.numpy as jnp

model_name = "google/flan-t5-small"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load Flax model for TPU
model = FlaxT5ForConditionalGeneration.from_pretrained(
    model_name,
    dtype=jnp.float32,
    _do_init=True
)

print(f"Model {model_name} loaded successfully for TPU!")
print(f"JAX devices: {jax.devices()}")
print(f"JAX device count: {jax.device_count()}")

# Setup W&B
os.environ["WANDB_API_KEY"] = "57b8585a9cdb363d54a7d215dd95c824d880868b"
wandb.login()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Model google/flan-t5-small loaded successfully!


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ohitit20 (ohitit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
def compute_logp_jax(params, prompt_ids, prompt_mask, target_ids, target_mask):
    """Compute log probability using JAX/Flax model"""
    # Prepare labels (replace pad tokens with -100)
    labels = jnp.where(target_mask, target_ids, -100)
    
    outputs = model(
        input_ids=prompt_ids,
        attention_mask=prompt_mask,
        labels=labels,
        params=params,
        train=True
    )
    return -outputs.loss

def ipo_loss_jax(logp_win, logp_lose, tau=0.05):
    """Compute IPO loss in JAX"""
    diff = logp_win - logp_lose - 0.5 / tau
    return jnp.mean(diff ** 2)

def tokenize_batch(questions, preferred, dispreferred, max_prompt_len=128, max_target_len=64):
    """Tokenize a batch of data"""
    batch_size = len(questions)
    
    # Prepare prompts
    prompts = [f"Generate a search query for: {q}\nQuery: " for q in questions]
    
    # Tokenize prompts
    prompt_encodings = tokenizer(
        prompts, 
        padding=True, 
        truncation=True, 
        max_length=max_prompt_len,
        return_tensors="np"
    )
    
    # Tokenize preferred and dispreferred completions
    preferred_encodings = tokenizer(
        preferred,
        padding=True,
        truncation=True,
        max_length=max_target_len,
        return_tensors="np"
    )
    
    dispreferred_encodings = tokenizer(
        dispreferred,
        padding=True,
        truncation=True,
        max_length=max_target_len,
        return_tensors="np"
    )
    
    return {
        'prompt_ids': jnp.array(prompt_encodings['input_ids']),
        'prompt_mask': jnp.array(prompt_encodings['attention_mask']),
        'preferred_ids': jnp.array(preferred_encodings['input_ids']),
        'preferred_mask': jnp.array(preferred_encodings['attention_mask']),
        'dispreferred_ids': jnp.array(dispreferred_encodings['input_ids']),
        'dispreferred_mask': jnp.array(dispreferred_encodings['attention_mask'])
    }

@jax.jit
def train_step(state, batch, tau=0.05):
    """Single training step compiled with JAX JIT"""
    def loss_fn(params):
        # Compute log probabilities for preferred completions
        logp_preferred = jax.vmap(
            lambda p_ids, p_mask, t_ids, t_mask: compute_logp_jax(params, p_ids[None], p_mask[None], t_ids[None], t_mask[None])
        )(batch['prompt_ids'], batch['prompt_mask'], batch['preferred_ids'], batch['preferred_mask'])
        
        # Compute log probabilities for dispreferred completions
        logp_dispreferred = jax.vmap(
            lambda p_ids, p_mask, t_ids, t_mask: compute_logp_jax(params, p_ids[None], p_mask[None], t_ids[None], t_mask[None])
        )(batch['prompt_ids'], batch['prompt_mask'], batch['dispreferred_ids'], batch['dispreferred_mask'])
        
        # Compute IPO loss
        loss = ipo_loss_jax(logp_preferred, logp_dispreferred, tau)
        return loss
    
    loss, grads = jax.value_and_grad(loss_fn)(state.params)
    state = state.apply_gradients(grads=grads)
    return state, loss

# Training setup
parent_path = '/content/drive/MyDrive/c438_project'
dataset_path = f'{parent_path}/preference_dataset_hotpotqa_final.json'
dataset = PreferenceDataset(dataset_path)

# Training hyperparameters
batch_size = 4  # Larger batch size for TPU efficiency
learning_rate = 5e-6
tau = 0.05
num_epochs = 3

# Initialize optimizer and training state
optimizer = optax.adamw(learning_rate=learning_rate)
rng = random.PRNGKey(42)
state = train_state.TrainState.create(
    apply_fn=model.__call__,
    params=model.params,
    tx=optimizer
)

print(f"Dataset size: {len(dataset)}")
print(f"Batch size: {batch_size}")
print(f"Starting IPO training on TPU...")

Dataset size: 70073
Starting IPO training...


In [ ]:
wandb.init(
    project="c438_project",
    name="ipo_training_tpu_run",
    config={
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "batch_size": batch_size,
        "model_name": model_name,
        "tau": tau,
        "device": "TPU",
        "device_count": jax.device_count()
    }
)

# Training loop
num_batches_per_epoch = len(dataset) // batch_size
rng = random.PRNGKey(42)

for epoch in range(num_epochs):
    total_loss = 0.0
    epoch_rng = random.fold_in(rng, epoch)
    
    pbar = tqdm(range(num_batches_per_epoch), desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch_idx in pbar:
        # Get batch
        batch_rng = random.fold_in(epoch_rng, batch_idx)
        raw_batch = dataset.get_batch(batch_size, batch_rng)
        
        # Extract data from batch
        questions = [item['question'] for item in raw_batch]
        preferred = [item['preferred'].strip() for item in raw_batch]
        dispreferred = [item['dispreferred'].strip() for item in raw_batch]
        
        # Tokenize batch
        batch = tokenize_batch(questions, preferred, dispreferred)
        
        # Training step
        try:
            state, loss = train_step(state, batch, tau)
            current_loss = float(loss)
            
            if jnp.isnan(loss):
                print(f"Skipping batch {batch_idx} due to NaN loss.")
                continue
            
            total_loss += current_loss
            avg_loss = total_loss / (batch_idx + 1)
            
            pbar.set_postfix({"loss": f"{avg_loss:.4f}"})
            wandb.log({"step_loss": current_loss, "average_loss": avg_loss})
            
        except Exception as e:
            print(f"Error in batch {batch_idx}: {e}")
            continue
    
    epoch_avg_loss = total_loss / num_batches_per_epoch
    print(f"[Epoch {epoch + 1}] Average Loss: {epoch_avg_loss:.4f}")
    wandb.log({"epoch": epoch + 1, "epoch_average_loss": epoch_avg_loss})
    
    # Save model checkpoint
    epoch_save_path = f"{parent_path}/ipo_trained_model_tpu/epoch_{epoch+1}"
    os.makedirs(epoch_save_path, exist_ok=True)
    
    # Save Flax model
    model.save_pretrained(
        epoch_save_path,
        params=state.params,
        push_to_hub=False
    )
    tokenizer.save_pretrained(epoch_save_path)
    print(f"Epoch {epoch + 1} model saved to {epoch_save_path}")

print("Training completed on TPU!")
wandb.finish()


Epoch 1/3:  27%|██▋       | 19181/70073 [57:57<2:34:43,  5.48it/s, loss=91.4975]

In [ ]:
def generate_query(question, max_length=30, temperature=0.8, top_p=0.9):
    """Generate a search query for the given question using the trained T5 model"""
    model.eval()

    # T5 works better with clear instruction format
    prompt = f"Generate a search query for this question: {question}"
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128)

    device = next(model.parameters()).device
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            early_stopping=False,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

    # For seq2seq models, decode the entire output (no need to slice)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up the generated text
    generated_text = generated_text.strip()

    # Remove question marks and periods
    generated_text = generated_text.replace('?', '').replace('.', '')


    return generated_text if generated_text else "search query"

# Test with example questions
test_questions = [
    "Who was the first person to climb Mount Everest?",
    "What is the capital of the country where the Eiffel Tower is located?",
    "Which movie won the Academy Award for Best Picture in 2020?",
    "What is the largest planet in our solar system?",
    "Who wrote the novel '1984'?"
]

print("Testing the trained model on example questions:")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")

    # Generate multiple queries with different parameters
    configs = [
        {"temperature": 0.5, "top_p": 0.8},
        {"temperature": 0.8, "top_p": 0.9},
        {"temperature": 1.0, "top_p": 0.95}
    ]

    for j, config in enumerate(configs):
        query = generate_query(question, **config)
        print(f"   Query {j+1}: {query}")

    print("-" * 40)

print("\nTesting completed!")